# Beaver Triples and multiplication

In this notebook we will show a tecnique to find the multiplication of two shared numbers using randomly generated shares.

We've seen that we can do multiplication in Shamir secret sharing only if we have $2t$ parties that can reconstruct the secret. In this case we will generate some random numbers called Beaver triples to help us perform the calculation. 

## Secretly computing one multiplication

In the first example we will try to secretly multipy two integers $x$ and $y$ that are owned by Ausiàs and Bernat, respectively. They want to compute $x*y$ without each other (or nobody else) know what is $x$ or $y$ actually. To this end we will use some computational parties $n$ that will carry the computation.

In [1]:
from random import seed, randrange
seed(3)

x = 10
y = 20

print(f"Ausias has a secret value ausias_secret={x}")
print(f"Bernat has a secret value bernat_secret={y}")

Ausias has a secret value ausias_secret=10
Bernat has a secret value bernat_secret=20


In [2]:
from crypt import GeneratePrimeGeneratorPair
from smpc import BeaverTriplesGenerator

n = 3 # number of parties involved
k = 1 # number of multiplications to perform
bits = 64 # bits of the calculation

p, _ = GeneratePrimeGeneratorPair(bits)

print(f"Multiplication will be performed using {n} parties")
print(f"Prime number of {bits} bits randomly chosen: {p}")

Multiplication will be performed using 3 parties
Prime number of 64 bits randomly chosen: 9628098341074011907


First thing Ausias and Bernat have to do is to split into additive shares their values:

In [3]:
def share(secret, n, p):
    # generate n random shares
    shares = [randrange(p) for _ in range(n-1)]
    shares.append((secret-sum(shares))%p)
    return shares

def reconstruct(shares, p):
    return (sum(shares)%p)

x_shares = share(x, n, p)
y_shares = share(y, n, p)

print(f"shares for Ausias value:\n\t{x_shares}")
print(f"shares for Bernat value:\n\t{y_shares}")

shares for Ausias value:
	[4567125481059076330, 8024907043293506373, 6664164157795441121]
shares for Bernat value:
	[9224214575254058992, 1257491421664474776, 8774490685229490066]


In [4]:
print(f"Ausias value reconstructed: {reconstruct(x_shares, p)}")
print(f"Bernat value reconstructed: {reconstruct(y_shares, p)}")

Ausias value reconstructed: 10
Bernat value reconstructed: 20


We have to generate the beaver triples, these are a triplet of shared values $[a]$, $[b]$, $[c]$ such that when reconstructed $c=a\times b$. Check the function BeaverTriplesGenerator for details on the generation of such numbers

In [5]:
generator = BeaverTriplesGenerator(n, p)
a, b, c = generator.GenerateKBatches(k)

print("Showing shares of a, b and c: \n")
print(f"[a] = {a}\n")
print(f"[b] = {b}\n")
print(f"[c] = {c}")

Showing shares of a, b and c: 

[a] = [281198838824332525, 5201432296111256459, 3303320766141227433]

[b] = [6948899913741526387, 6793657391693877890, 6673149354162615119]

[c] = [7820751555456555897, 1834301485405493978, 7913907833595222854]


Let's check that the triple generation is done correctly by checking that:

$$a*b = c$$

so we reconstruct a, b and c modulo p and then check the equality

In [6]:
a_reconstructed = reconstruct(a, p)
b_reconstructed = reconstruct(b, p)
c_reconstructed = reconstruct(c, p)

assert a_reconstructed*b_reconstructed%p==c_reconstructed%p
print(f"cumulative sum of a times cumulative sum of b modulo p {a_reconstructed*b_reconstructed%p}")
print(f"cumulative sum of c modulo p {c_reconstructed%p}")
print("They are equal!")

cumulative sum of a times cumulative sum of b modulo p 7940862533383260822
cumulative sum of c modulo p 7940862533383260822
They are equal!


The shares $[a]$, $[b]$, $[c]$ have to be split into the workers. We also print the shared value of $x$ and $y$ to show what each party gets.

In [7]:
for i, (x_, y_, a_, b_, c_) in enumerate(zip(x_shares, y_shares, a, b, c)):
    print(f"Party {i}:\n\tx={x_}\n\ty={y_}\n\ta={a_}\n\tb={b_}\n\tc={c_}\n\t")

Party 0:
	x=4567125481059076330
	y=9224214575254058992
	a=281198838824332525
	b=6948899913741526387
	c=7820751555456555897
	
Party 1:
	x=8024907043293506373
	y=1257491421664474776
	a=5201432296111256459
	b=6793657391693877890
	c=1834301485405493978
	
Party 2:
	x=6664164157795441121
	y=8774490685229490066
	a=3303320766141227433
	b=6673149354162615119
	c=7913907833595222854
	


Now each party has to calculate:

$$[\epsilon]=[x]-[a]$$
$$[\delta]=[y]-[b]$$

In [8]:
e, d = [], []

for i, (x_, y_, a_, b_, c_) in enumerate(zip(x_shares, y_shares, a, b, c)):
    e.append(x_ - a_)
    d.append(y_ - b_)
    
epsilon = reconstruct(e, p)
delta = reconstruct(d, p)

print(f"They open epsilon and delta: \n\nepsilon={epsilon}\ndelta={delta}")

They open epsilon and delta: 

epsilon=842146439997195500
delta=8468588363624016345


Recall that $\epsilon$ and $\delta$ are one time padded (we use a different random beaver triple every time we run a multiplication) and therefeore do not provide any information about $x$ and $y$ the values that Ausias and Bernat want to multiply using third party helpers.

Now each party has to calculate:

$$[z] = [c] + \epsilon [b] + \delta[a] + \epsilon\delta$$

In [9]:
z = []
for x_, y_, a_, b_, c_ in zip(x_shares, y_shares, a, b, c):
    z_ = (c_ + delta*x_%p + epsilon*y_%p - epsilon*delta%p)%p
    z.append(z_)

In [10]:
print(reconstruct(z, p))

2506911905567605917


In [11]:
print(x*y, reconstruct(z, p))
assert x*y==reconstruct(z, p), "Something went wrong, reconstruction was not correct"

200 2506911905567605917


AssertionError: Something went wrong, reconstruction was not correct

In [12]:
zp = reconstruct(c, p) + epsilon*reconstruct(b, p)%p + delta*reconstruct(a, p)%p + epsilon*delta%p
print(zp%p)

200


In [13]:
zp = reconstruct(c, p) + delta*x%p + epsilon*y%p - epsilon*delta%p
print(zp%p)

200
